In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType

from delta import *

In [2]:
spark = ( 
    SparkSession
    .builder
    .master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate() 
)

25/04/16 09:52:51 WARN Utils: Your hostname, bonetti-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/04/16 09:52:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/bonetti/.cache/pypoetry/virtualenvs/atividade-engenharia-de-dados-pyspark-oL9R8c7J-py3.12/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/bonetti/.ivy2/cache
The jars for the packages stored in: /home/bonetti/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e92c3b36-67f7-45d0-8a37-21362ee5420c;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 143ms :: artifacts dl 11ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0 

In [3]:
spark

In [4]:
data = [
    ("1", "PRODUTO_X","MARCA_X", "CATEGORIA_X" ,800.00, 50),
    ("2", "PRODUTO_Y", "MARCA_Y", "CATEGORIA_Y", 500.00, 100),
    ("3", "PRODUTO_Z","MARCA_Z", "CATEGORIA_Z", 150.00, 150)
]

schema = (
    StructType([
        StructField("ID_PRODUTO", StringType(),True),
        StructField("NOME_PRODUTO", StringType(),True),
        StructField("MARCA_PRODUTO", StringType(),True),
        StructField("CATEGORIA_PRODUTO", StringType(),True),
        StructField("PRECO_PRODUTO", FloatType(), True),
        StructField("QUANTIDADE_PRODUTO", IntegerType(), True)
    ])
)

df = spark.createDataFrame(data=data,schema=schema)

df.show(truncate=False)

+----------+------------+-------------+-----------------+-------------+------------------+
|ID_PRODUTO|NOME_PRODUTO|MARCA_PRODUTO|CATEGORIA_PRODUTO|PRECO_PRODUTO|QUANTIDADE_PRODUTO|
+----------+------------+-------------+-----------------+-------------+------------------+
|1         |PRODUTO_X   |MARCA_X      |CATEGORIA_X      |800.0        |50                |
|2         |PRODUTO_Y   |MARCA_Y      |CATEGORIA_Y      |500.0        |100               |
|3         |PRODUTO_Z   |MARCA_Z      |CAATEGORIA_Z     |150.0        |150               |
+----------+------------+-------------+-----------------+-------------+------------------+



In [5]:
( 
    df
    .write
    .format("delta")
    .mode('overwrite')
    .save("./RAW/PRODUTOS")
)

25/04/16 09:53:09 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

In [6]:
new_data = [
    ("4","PRODUTO_A", "MARCA_A", "CATEGORIA_A", 50.00, 60),
    ("5","PRODUTO_B", "MARCA_B", "CATEGORIA_B", 150.00, 90),
    ("6","PRODUTO_C", "MARCA_C", "CATEGORIA_C", 300.00, 20)
]

df_new = spark.createDataFrame(data=new_data, schema=schema)

df_new.show()

+----------+------------+-------------+-----------------+-------------+------------------+
|ID_PRODUTO|NOME_PRODUTO|MARCA_PRODUTO|CATEGORIA_PRODUTO|PRECO_PRODUTO|QUANTIDADE_PRODUTO|
+----------+------------+-------------+-----------------+-------------+------------------+
|         4|   PRODUTO_A|      MARCA_A|      CATEGORIA_A|         50.0|                60|
|         5|   PRODUTO_B|      MARCA_B|      CATEGORIA_B|        150.0|                90|
|         6|   PRODUTO_C|      MARCA_C|      CATEGORIA_C|        300.0|                20|
+----------+------------+-------------+-----------------+-------------+------------------+



In [7]:
deltaTable = DeltaTable.forPath(spark, "./RAW/PRODUTOS")

(
    deltaTable.alias("dados_atuais")
    .merge(
        df_new.alias("novos_dados"),
        "dados_atuais.ID_PRODUTO = novos_dados.ID_PRODUTO"
    )
    .whenMatchedUpdateAll()
    .whenNotMatchedInsertAll()
    .execute()
)


In [10]:
deltaTable.delete("ID_PRODUTO = 4")

In [11]:
(
    deltaTable
    .history()
    .select("version", "timestamp", "operation", "operationMetrics")
    .show()
)

+-------+--------------------+---------+--------------------+
|version|           timestamp|operation|    operationMetrics|
+-------+--------------------+---------+--------------------+
|      2|2025-04-16 09:54:...|   DELETE|{numRemovedFiles ...|
|      1|2025-04-16 09:53:...|    MERGE|{numTargetRowsCop...|
|      0|2025-04-16 09:53:...|    WRITE|{numFiles -> 3, n...|
+-------+--------------------+---------+--------------------+



In [12]:
(
    spark
    .read
    .format('delta')
    .option("versionAsOf", 2)
    .load('./RAW/PRODUTOS')
    .show()
)

+----------+------------+-------------+-----------------+-------------+------------------+
|ID_PRODUTO|NOME_PRODUTO|MARCA_PRODUTO|CATEGORIA_PRODUTO|PRECO_PRODUTO|QUANTIDADE_PRODUTO|
+----------+------------+-------------+-----------------+-------------+------------------+
|         3|   PRODUTO_Z|      MARCA_Z|     CAATEGORIA_Z|        150.0|               150|
|         1|   PRODUTO_X|      MARCA_X|      CATEGORIA_X|        800.0|                50|
|         2|   PRODUTO_Y|      MARCA_Y|      CATEGORIA_Y|        500.0|               100|
|         5|   PRODUTO_B|      MARCA_B|      CATEGORIA_B|        150.0|                90|
|         6|   PRODUTO_C|      MARCA_C|      CATEGORIA_C|        300.0|                20|
+----------+------------+-------------+-----------------+-------------+------------------+



In [13]:
deltaTable.update("ID_PRODUTO = 2", set = { "PRECO_PRODUTO": "400.00" })

In [19]:
(
    spark
    .read
    .format('delta')
    .load('./RAW/PRODUTOS')
    .show()
)

+----------+------------+-------------+-----------------+-------------+------------------+
|ID_PRODUTO|NOME_PRODUTO|MARCA_PRODUTO|CATEGORIA_PRODUTO|PRECO_PRODUTO|QUANTIDADE_PRODUTO|
+----------+------------+-------------+-----------------+-------------+------------------+
|         3|   PRODUTO_Z|      MARCA_Z|     CAATEGORIA_Z|        150.0|               150|
|         1|   PRODUTO_X|      MARCA_X|      CATEGORIA_X|        800.0|                50|
|         2|   PRODUTO_Y|      MARCA_Y|      CATEGORIA_Y|        400.0|               100|
|         5|   PRODUTO_B|      MARCA_B|      CATEGORIA_B|        150.0|                90|
|         6|   PRODUTO_C|      MARCA_C|      CATEGORIA_C|        300.0|                20|
+----------+------------+-------------+-----------------+-------------+------------------+

